In [1]:
import cffi
import numba
# import numba.core.typing.cffi_utils as cffi_support
import numpy as np

import ufl
# from dolfinx.cpp.fem import Form_float64
from dolfinx.jit import ffcx_jit
from dolfinx import fem, mesh, io

from mpi4py import MPI
from petsc4py import PETSc

import ctypes
import ctypes.util
import petsc4py.lib
from mpi4py import MPI
from petsc4py import get_config as PETSc_get_config

In [2]:
# Get details of PETSc install
petsc_dir = PETSc_get_config()['PETSC_DIR']
petsc_arch = petsc4py.lib.getPathArchPETSc()[1]

# Get PETSc int and scalar types
scalar_size = np.dtype(PETSc.ScalarType).itemsize
index_size = np.dtype(PETSc.IntType).itemsize

if index_size == 8:
    c_int_t = "int64_t"
    ctypes_index = ctypes.c_int64
elif index_size == 4:
    c_int_t = "int32_t"
    ctypes_index = ctypes.c_int32
else:
    raise RuntimeError(f"Cannot translate PETSc index size into a C type, index_size: {index_size}.")

if scalar_size == 8:
    c_scalar_t = "double"
    numba_scalar_t = numba.types.float64
elif scalar_size == 4:
    c_scalar_t = "float"
    numba_scalar_t = numba.types.float32
else:
    raise RuntimeError(
        f"Cannot translate PETSc scalar type to a C type, complex: {complex} size: {scalar_size}.")

# Load PETSc library via ctypes
petsc_lib_name = ctypes.util.find_library("petsc")
if petsc_lib_name is not None:
    petsc_lib_ctypes = ctypes.CDLL(petsc_lib_name)
else:
    try:
        petsc_lib_ctypes = ctypes.CDLL(os.path.join(petsc_dir, petsc_arch, "lib", "libpetsc.so"))
    except OSError:
        petsc_lib_ctypes = ctypes.CDLL(os.path.join(petsc_dir, petsc_arch, "lib", "libpetsc.dylib"))
    except OSError:
        print("Could not load PETSc library for CFFI (ABI mode).")
        raise

# Get the PETSc MatSetValuesLocal function via ctypes
MatSetValues_ctypes = petsc_lib_ctypes.MatSetValuesLocal
MatSetValues_ctypes.argtypes = (ctypes.c_void_p, ctypes_index, ctypes.POINTER(
    ctypes_index), ctypes_index, ctypes.POINTER(ctypes_index), ctypes.c_void_p, ctypes.c_int)
del petsc_lib_ctypes

In [3]:
ffi = cffi.FFI()

def get_kernel(domain, form):
    ufcx_form, _, _ = ffcx_jit(domain.comm, form, form_compiler_params={"scalar_type": "double"})
    kernel = ufcx_form.integrals(0)[0].tabulate_tensor_float64
    return kernel

In [4]:
domain = mesh.create_unit_square(MPI.COMM_WORLD, 1, 1)

In [5]:
# V = fem.VectorFunctionSpace(domain, ("Lagrange", 1))
# u, v = ufl.TrialFunction(V), ufl.TestFunction(V)

# lambda_ = 1
# mu_ = 1
# C = np.array([[lambda_ + 2*mu_, lambda_, 0],
#               [lambda_, lambda_ + 2*mu_, 0],
#               [0, 0, 2*mu_]], dtype=PETSc.ScalarType)
# C_const = fem.Constant(domain, C)

# coeff1 = fem.Function(fem.FunctionSpace(domain, ('DG', 0)))
# coeff2 = fem.Function(fem.FunctionSpace(domain, ('DG', 0)))
# coeff1.vector.set(1)
# coeff2.vector.set(1)

# def epsilon(u):
#     return ufl.sym(ufl.grad(u))

# def as_2D_tensor(X):
#     return ufl.as_tensor([[X[0], X[2]],
#                           [X[2], X[1]]])

# def sigma(u):
#     eps = epsilon(u)
#     return as_2D_tensor(C_const * ufl.as_vector([eps[0,0], eps[1,1], eps[0,1]]))

# a = ufl.inner( coeff1 * sigma(u), epsilon(v) * coeff2 ) * ufl.dx

# get_kernel(domain, a)

In [6]:
# class CustomFunction(fem.Function):
#     def __init__(self,):
        

In [7]:
import sys
sys.path.append("../")
import fenicsx_support

In [8]:
N = 1
domain = mesh.create_unit_square(MPI.COMM_WORLD, N, N)#, mesh.CellType.quadrilateral

deg, q_deg = 1, 2
V = fem.VectorFunctionSpace(domain, ("P", deg))

# quadrature elements and function spaces
QV = ufl.VectorElement(
    "Quadrature", domain.ufl_cell(), q_deg, quad_scheme="default", dim=3
)
QT = ufl.TensorElement(
    "Quadrature",
    domain.ufl_cell(),
    q_deg,
    quad_scheme="default",
    shape=(3, 3),
)
VQV = fem.FunctionSpace(domain, QV)
VQT = fem.FunctionSpace(domain, QT)

# define functions
u_, du = ufl.TestFunction(V), ufl.TrialFunction(V)
u = fem.Function(V)
q_sigma = fem.Function(VQV)
q_dsigma = fem.Function(VQT)

num_dofs_global = V.dofmap.index_map.size_global * V.dofmap.index_map_bs
num_gauss_local = len(q_sigma.x.array[:]) // 3
num_gauss_global = domain.comm.reduce(num_gauss_local, op=MPI.SUM, root=0)

# define form
dxm = ufl.dx(metadata={"quadrature_degree": q_deg, "quadrature_scheme": "default"})

# @numba.njit()
def eps(u):
    e = ufl.sym(ufl.grad(u))
    return ufl.as_vector((e[0, 0], e[1, 1], 2 * e[0, 1]))

R = ufl.inner(eps(u_), q_sigma) * dxm
dR = ufl.inner(eps(du), ufl.dot(q_dsigma, eps(u_))) * dxm

E, nu = 20000, 0.3

# Hookes law for plane stress
C11 = E / (1.0 - nu * nu)
C12 = C11 * nu
C33 = C11 * 0.5 * (1.0 - nu)
C = np.array([[C11, C12, 0.0], [C12, C11, 0.0], [0.0, 0.0, C33]], dtype=PETSc.ScalarType)

# def evaluate_constitutive_law(u):

#     # prepare strain evaluation
#     map_c = domain.topology.index_map(domain.topology.dim)
#     num_cells = map_c.size_local + map_c.num_ghosts
#     cells = np.arange(0, num_cells, dtype=np.int32)

#     basix_celltype = getattr(basix.CellType, domain.topology.cell_type.name)
#     q_points, wts = basix.make_quadrature(basix_celltype, q_deg)
#     strain_expr = fem.Expression(eps(u), q_points)

#     # Actually compute a strain matrix containing one row per cell.
#     strains = strain_expr.eval(cells)
#     assert strains.shape[0] == num_cells
#     assert strains.shape[1] == len(q_points) * 3

#     # here _could_ be a loop over all quadrature points in
#     # c++, mfront, numba, ...
#     # For this simple case, we can evaluate it as one matrix operation.
#     strain_matrix = strains.reshape((-1, 3))
#     n_gauss = len(strain_matrix)

#     q_sigma.x.array[:] = (strain_matrix @ C).flatten()
#     q_dsigma.x.array[:] = np.tile(C.flatten(), n_gauss)


r"""
Set up 

  +---------+
/||         |->
/||         |->
/||         |-> u_bc
  o---------+
 / \
-----
"""


def left(x):
    return np.isclose(x[0], 0.0)


def right(x):
    return np.isclose(x[0], 1.0)


def origin(x):
    return np.logical_and(np.isclose(x[0], 0.0), np.isclose(x[1], 0.0))


u_bc = fem.Constant(domain, 0.0)  # expression for boundary displacement

dim = domain.topology.dim - 1
b_facets_l = mesh.locate_entities_boundary(domain, dim, left)
b_facets_r = mesh.locate_entities_boundary(domain, dim, right)
b_facets_o = mesh.locate_entities_boundary(domain, dim - 1, origin)

b_dofs_l = fem.locate_dofs_topological(V.sub(0), dim, b_facets_l)
b_dofs_r = fem.locate_dofs_topological(V.sub(0), dim, b_facets_r)
b_dofs_o = fem.locate_dofs_topological(V.sub(1), dim - 1, b_facets_o)

bcs = [
    fem.dirichletbc(PETSc.ScalarType(0), b_dofs_l, V.sub(0)),
    fem.dirichletbc(u_bc, b_dofs_r, V.sub(0)),
    fem.dirichletbc(PETSc.ScalarType(0), b_dofs_o, V.sub(1)),
]


In [20]:
print(q_sigma.x.array.shape[0], q_dsigma.x.array.shape, u.x.array.shape)

18 (54,) (8,)


In [10]:
dR.coefficients()

(Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 1), TensorElement(FiniteElement('Quadrature', triangle, 2, quad_scheme='default'), shape=(3, 3), symmetry={})), 139689571268800),)

In [11]:
R.coefficients()

(Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 1), VectorElement(FiniteElement('Quadrature', triangle, 2, quad_scheme='default'), dim=3)), 139689706377344),)

In [12]:
# Unpack mesh and dofmap data
num_owned_cells = domain.topology.index_map(domain.topology.dim).size_local
num_cells = num_owned_cells + domain.topology.index_map(domain.topology.dim).num_ghosts
x_dofs = domain.geometry.dofmap.array.reshape(num_cells, 3)
x = domain.geometry.x
dofmap = V.dofmap.list.array.reshape(num_cells, 3).astype(np.dtype(PETSc.IntType))

In [52]:
A = fem.petsc.create_matrix(fem.form(dR))
A.zeroEntries()
A.assemble()
# print(A[:,:])
b = fem.petsc.create_vector(fem.form(R))
# b4 = fem.Function(V)
# b = b4.x.array

with b.localForm() as b_local:
    b_local.set(0.0)

u = fem.Function(V)

map_c = domain.topology.index_map(domain.topology.dim)
num_cells = map_c.size_local + map_c.num_ghosts
# cells = np.arange(0, num_cells, dtype=np.int32)
N_dofs_element = V.element.space_dimension
N_sub_spaces = V.num_sub_spaces # V.dofmap.index_map_bs
dofmap = V.dofmap.list.array.reshape(num_cells, int(N_dofs_element/N_sub_spaces))

dofmap_tmp = (N_sub_spaces*np.repeat(dofmap, N_sub_spaces).reshape(-1, N_sub_spaces) + np.arange(N_sub_spaces)).reshape(-1, N_dofs_element).astype(np.dtype(PETSc.IntType))        

kernel_A = get_kernel(domain, dR)
kernel_b = get_kernel(domain, R)

# eps_calculated = fem.Function(VQV)
eps_calculated = fenicsx_support.interpolate_quadrature(eps(u), q_deg, domain)
print(eps_calculated.shape) # num_cells * len(eps) * len(n_gauls_on_element)
strain_matrix = eps_calculated.reshape((-1, 3))
n_gauss = len(strain_matrix) #global in the domain

q_sigma.x.array[:] = (strain_matrix @ C).flatten()
q_dsigma.x.array[:] = np.tile(C.flatten(), n_gauss)

q_sigma_values = q_sigma.x.array[:] 
q_dsigma_values = np.tile(C.flatten(), n_gauss)

print(q_dsigma_values.shape, q_sigma_values.shape)

@numba.njit
def assemble_ufc(A, b, mesh, dofmap, num_cells, set_vals, mode):
    v, x = mesh
    entity_local_index = np.array([0], dtype=np.intc)
    perm = np.array([0], dtype=np.uint8)
    geometry = np.zeros((3, 3))
    coeffs_A = np.full(27, 1, dtype=PETSc.ScalarType) ####!!!!!!!!!!!!!!!1
    coeffs_b = np.zeros((9), dtype=PETSc.ScalarType) #!!!!!!!!!!!
    constants = C #np.array(C, dtype=PETSc.ScalarType)

    b_local = np.zeros(N_dofs_element, dtype=PETSc.ScalarType)
    A_local = np.zeros((N_dofs_element, N_dofs_element), dtype=PETSc.ScalarType)
    
    for cell in range(num_cells):
        pos = rows = cols = dofmap[cell, :]

        # for j in range(3):
        geometry[:] = x[v[cell, :], :]
        b_local.fill(0.)
        A_local.fill(0.)

        coeffs_b = q_sigma_values.reshape(2, -1)[cell]
        coeffs_A = q_dsigma_values.reshape(2, -1)[cell]

        kernel_b(ffi.from_buffer(b_local), ffi.from_buffer(coeffs_b),
                 ffi.from_buffer(constants),
                 ffi.from_buffer(geometry), ffi.from_buffer(entity_local_index),
                 ffi.from_buffer(perm))
        b[pos] += b_local
        
        kernel_A(ffi.from_buffer(A_local), ffi.from_buffer(coeffs_A),
               ffi.from_buffer(constants),
               ffi.from_buffer(geometry), ffi.from_buffer(entity_local_index),
               ffi.from_buffer(perm))
        # print('\n', A_local)

        set_vals(A, N_dofs_element, rows.ctypes, N_dofs_element, cols.ctypes, A_local.ctypes, mode)
    

solver = PETSc.KSP().create(domain.comm)
solver.setType("preonly")
solver.getPC().setType("lu")

f = io.XDMFFile(domain.comm, "displacements.xdmf", "w", encoding=io.XDMFFile.Encoding.HDF5)
f.write_mesh(domain)


(2, 9)
(54,) (18,)


In [55]:
u_bc_max = 42.0
for t in np.linspace(0.0, 1.0, 5):
    # update value of Dirichlet BC
    u_bc.value = t * u_bc_max

    # update matrix (pointless in linear elasticity...)
    A.zeroEntries()
    # fem.petsc.assemble_matrix(A, dR, bcs=bcs)

    # update residual vector
    with b.localForm() as b_local:
        b_local.set(0.0)
        
    assemble_ufc(A.handle, b.array, (x_dofs, x), dofmap_tmp, num_owned_cells, MatSetValues_ctypes, PETSc.InsertMode.ADD_VALUES)
    A.assemble()
    # fem.petsc.assemble_vector(b, fem.form(R))

    fem.apply_lifting(b, [fem.form(dR)], bcs=[bcs], x0=[u.vector], scale=-1.0)
    b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
    fem.set_bc(b, bcs, u.vector, -1.0)

    solver.setOperators(A)

    # Solve for the displacement increment du, apply it and udpate ghost values
    du = fem.Function(V)  # Should be outside of loop, instructive here.
    solver.solve(b, du.vector)
    u.x.array[:] -= du.x.array[:]
    u.x.scatter_forward()

    # post processing
    # check_solution(u, t * u_bc_max)
    f.write_function(u, t)

Error: error code 73
[0] KSPSolve() at /usr/local/petsc/src/ksp/ksp/interface/itfunc.c:1086
[0] KSPSolve_Private() at /usr/local/petsc/src/ksp/ksp/interface/itfunc.c:852
[0] KSPSetUp() at /usr/local/petsc/src/ksp/ksp/interface/itfunc.c:408
[0] PCSetUp() at /usr/local/petsc/src/ksp/pc/interface/precon.c:1017
[0] PCSetUp_LU() at /usr/local/petsc/src/ksp/pc/impls/factor/lu/lu.c:95
[0] MatLUFactorSymbolic() at /usr/local/petsc/src/mat/interface/matrix.c:3131
[0] MatLUFactorSymbolic_SeqAIJ() at /usr/local/petsc/src/mat/impls/aij/seq/aijfact.c:308
[0] Object is in wrong state
[0] Matrix is missing diagonal entry 0

In [22]:
q_dsigma_values.s

array([21978.02197802,  6593.40659341,     0.        ,  6593.40659341,
       21978.02197802,     0.        ,     0.        ,     0.        ,
        7692.30769231, 21978.02197802,  6593.40659341,     0.        ,
        6593.40659341, 21978.02197802,     0.        ,     0.        ,
           0.        ,  7692.30769231, 21978.02197802,  6593.40659341,
           0.        ,  6593.40659341, 21978.02197802,     0.        ,
           0.        ,     0.        ,  7692.30769231, 21978.02197802,
        6593.40659341,     0.        ,  6593.40659341, 21978.02197802,
           0.        ,     0.        ,     0.        ,  7692.30769231,
       21978.02197802,  6593.40659341,     0.        ,  6593.40659341,
       21978.02197802,     0.        ,     0.        ,     0.        ,
        7692.30769231, 21978.02197802,  6593.40659341,     0.        ,
        6593.40659341, 21978.02197802,     0.        ,     0.        ,
           0.        ,  7692.30769231])

In [15]:
q_dsigma

Coefficient(FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 1), TensorElement(FiniteElement('Quadrature', triangle, 2, quad_scheme='default'), shape=(3, 3), symmetry={})), 139689571268800)

In [16]:
q_sigma.x.array.shape

(18,)

In [17]:


def check_solution(u, u_bc_value):
    """
    Defines a grid of test points xs and compares the FE values u(xs)
    to the analytic displacement solution.

    ux(x,y) =       x * u_bc_value
    uy(x,y) = -nu * y * u_bc_value

    """

    def eval_function_at_points(f, points):
        """
        Evaluates `f` at `points`. Adapted from
        https://jorgensd.github.io/df-tutorial/chapter1/membrane_code.html
        """
        mesh = f.function_space.mesh
        tree = df.geometry.BoundingBoxTree(mesh, mesh.geometry.dim)
        cell_candidates = df.geometry.compute_collisions(tree, points)
        colliding_cells = df.geometry.compute_colliding_cells(
            mesh, cell_candidates, points
        )

        points_on_proc = []
        cells = []
        for i, point in enumerate(points):
            if len(colliding_cells.links(i)) > 0:
                points_on_proc.append(point)
                cells.append(colliding_cells.links(i)[0])
        points_on_proc = np.array(points_on_proc, dtype=np.float64)
        return f.eval(points_on_proc, cells), points_on_proc

    x_check = np.linspace(0, 1, 5)
    x, y = np.meshgrid(x_check, x_check)
    xs = np.vstack([x.flat, y.flat, np.zeros(len(x.flat))]).T
    u_fem, xs = eval_function_at_points(u, xs)

    # analytic solution
    u_ref = np.array([xs[:, 0] * u_bc_value, -nu * xs[:, 1] * u_bc_value]).T
    assert np.linalg.norm(u_fem - u_ref) < 1.0e-10


# Here, we manually build PETSc objects and simulate a quasistaic loading.
# Classes like df.fem.petsc.LinearProblem would trivialize that step. A look
# "under the hood" is quite instructive, though:
b = df.fem.petsc.create_vector(R)
A = df.fem.petsc.create_matrix(dR)

solver = PETSc.KSP().create(mesh.comm)
solver.setType("preonly")
solver.getPC().setType("lu")

f = df.io.XDMFFile(mesh.comm, "displacements.xdmf", "w")
f.write_mesh(mesh)


u_bc_max = 42.0
for t in np.linspace(0.0, 1.0, 5):
    # update value of Dirichlet BC
    u_bc.value = t * u_bc_max

    print0(f"Solving {t = :6.3f} with {u_bc.value = :6.3f}...")

    evaluate_constitutive_law(u)

    # update matrix (pointless in linear elasticity...)
    A.zeroEntries()
    df.fem.petsc.assemble_matrix(A, dR, bcs=bcs)
    A.assemble()

    # update residual vector
    with b.localForm() as b_local:
        b_local.set(0.0)
    df.fem.petsc.assemble_vector(b, R)

    df.fem.apply_lifting(b, [dR], bcs=[bcs], x0=[u.vector], scale=-1.0)
    b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
    df.fem.set_bc(b, bcs, u.vector, -1.0)

    solver.setOperators(A)

    # Solve for the displacement increment du, apply it and udpate ghost values
    du = df.fem.Function(V)  # Should be outside of loop, instructive here.
    solver.solve(b, du.vector)
    u.x.array[:] -= du.x.array[:]
    u.x.scatter_forward()

    # post processing
    check_solution(u, t * u_bc_max)
    f.write_function(u, t)


NameError: name 'df' is not defined